<a href="https://colab.research.google.com/github/roklp/MLP34/blob/main/240111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 문제 상황
- 편의점 감튀(130g) 근데 실제로 재보니 122.02g임
- 어이가없어서 2주간 매일 사서 무게측정했음
 + 14개의 표본의 평균 계산했는데, 128.451g 나옴 > 항의 입갤

## 답

- 사용 통계 검정 방법
 + t-test, One Sample T-Test
 + 표본 14개 ==> 정규성 검정 진행 후, 정규성 검정 통과 할 떄, One Sample T-Test

- 귀무가설과 대립가설 설정
 + 귀무가설 : 모평균(130g)과 표본평균 (128.451g)과 같다
 + 대립가설 : 모평균(130g)과 표본평균 (128.451g)과 같지 않다 (양측검정)
 + 대립가설 : 모평균(130g)> 표본평균 (128.451g)보다 크다 (단측검정)
- 교재 302pg 확인

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

In [3]:
!pip install scipy

In [26]:
%precision 3
np.random.seed(1111)

In [4]:
import scipy
scipy.__version__

'1.11.4'

In [11]:
df = pd.read_csv('/content/drive/MyDrive/data/ch11_potato.csv')
sample = np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [12]:
s_mean = np.mean(sample)
s_mean

128.4507142857143

In [ ]:
# 표본집단이 정규분포를 따르고 있다고 하면 "정규성 검정"을 할 필요가 없다
# 따라서 비모수 검정을 쓸 필요가 없고 모수 검정을 하면 된다

In [15]:
rv = stats.norm(130, # 모평균
                np.sqrt(9/14)) # 9 모분산, 14 표본의 갯수
rv.isf(0.95) # 0.95 신뢰구간 95%

128.68118313069039

## 검정통계량

In [16]:
# 모평균 130
z = (s_mean - 130) / np.sqrt(9/14)
z

-1.932298779026813

In [17]:
# 검정 통계랑에 관한 임곗값을 구하기

rv = stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [18]:
rv.cdf(z) # 유의수준 0.05보다 작음

# 귀무가설 : 모평균 130g 표본평균 128.451g과 같다
# 대립가설 : 모평균 130g 표본평균 128.451g과 다르다

# 대립가설 채탁을 한다는 말 << 통계에는 없음
# 귀무가설 채택 or 귀무가설 기각(=대립가설 채택)

0.026661319523126635

In [20]:
# 양측검정

z = (s_mean - 130) / np.sqrt(9/14)
rv = stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

In [21]:
rv.cdf(z) * 2

0.05332263904625327

## 질문
- 단측검정, 양측검정을 어떤 상황에서 구분해서 사용하나요?
  + 양측검정 A/B가 같지않을때
   -  해석 : 우리가 모두 아는 평균 가지고 비교 가능

  + 단측검정 A > B, B > A

## 가설 검정의 두 가지 오류
- 제 1종 오류: 귀무가설이 옳을 때 귀무가설을 기각하는 오류
- 제 2종 오류: 대립가설이 옳을 때 귀무가설을 채택하는 오류

## 제 1종 오류
- 실제로 평균이 130g인데도 불구하고 평균은 130g보다 작다라고 결론을 내리는 상황
 + False Positive(오탐)


In [29]:
rv = stats.norm(130, 3)
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.051

- 1종 오류를 범할 비율은 0.053 / 약 5%의 비율로 130g보다 작다라고 잘못 탐지하는 것과 같음
- 1종 오류를 범할 확률을 위험률 이라고 함
- 좀 더 엄격하게 적용하고 싶다고 하면 유의수준 1%에서 가설검정을 수행

### 제 2종 오류
- 표본평균이 > 130g 임에도 불구하고 130g 보다 작다라는 결론을 얻을 수 없는 상황
  + 본래 검출해야 하는 것을 검출하지 못했기때문에 미탐 false negative

In [31]:
rv = stats.norm(128, 3)

In [34]:
rv = stats.norm(128, 3)
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1

cnt / n_samples

0.195

- 0,197의 의미는 검정력 / 제대로 탐지하지 못할 확률이 20% 되더라
- 제 2종 오류는 언제나 모집단의 정보에 의존함
   + 중요한 포인트는 분석가는 모집단의 구체적인 정보를 알 방법이 없음

### 모분산을 안다
- 현실적으로 모집단 아는건 불가능함 ㅇㅇ
  + 건강보험과 통계 / 전국민 데이터가 있는 자료(국가)
- 모분산을 알고 있다는 뜻은 모집단을 알고 있다 > 모평균도 구할 수 있음 > 모평균에 대한 검정을 우리가 굳이 할 필요 없음

In [39]:
def pmean_test(sample, mean0, alpha=0.05):
  '''
  smaple : 표본 샘플 데이터
  mean0 : 모평균을 제안 (감자칩의 무게와 같이 이미 기 제안이 된 것)
  alpha : 신뢰구간 95% 수준의 신뢰구간

  '''
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    print(f't통계랑 값은 {t:.3f}')
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

IndentationError: unexpected indent (<ipython-input-39-bc035cac5ecc>, line 8)

In [38]:
pmean_test(sample, 130)

귀무가설을 채택
p값은 0.169


## 비전공자 / 이공계
- 비전공자는 해당되는 수식을 일반인에게 어떻게 풀어서 쉽게 설명을 해줄 것인가가 포인트

- 이공계는 수식을 잘 정리를 해서 블로그나 깃허브에 올리기

### z 통계량 / t 통계량
- 모집단을 알고, 모평균을 알고, 모분산을 다 알고잇다.= z-검정
  + z - 검정 : z - 통계량

- 표본을 추출한다, 표본 평균을 알고 표본 분산을 알고 있다.
  + t-검정 : t-통계량


# 2표본 문제에 관한 가설검정
- 대응표본 : 정규분포를 가정할 수 있음 (정규성 검정 통과)
  + 정규성 검정 통과 : 대응표본 t-검정
  + 정규성 검정 통과 실패 : 비모수 검정, 윌콕슨의 부호순위검정

- 독립표본 : 정규분포를 가정할 수 있음
 + 정규성 검정 통과 : 독립비교 t-검정
 + 정규성 검정 통과 실패 : 맨-위트니의 U검정




### 대응표본 t-검정
- A 학생은 친구 20명에게 1주일간 근력 운동을 하게 함 (intervention, 개입)
- 운동 전후에 집중력을 측정하는 테스트 받게 됨

 - 귀무가설: 근력 운동은 집중력에 영향을 미치지 않았다.
 - 대립가설: 근력 운동은 집중력을 향상시켰다

In [2]:
import pandas as pd

training_rel = pd.read_csv('/content/drive/MyDrive/data/ch11_training_rel.csv')

training_rel

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37
6,55,60
7,49,52
8,49,68
9,52,70


In [7]:
from scipy import stats

training_rel = pd.read_csv('/content/drive/MyDrive/data/ch11_training_rel.csv')

training_rel

t, p = stats.ttest_lsamp(training_rel['차'], 0)

t, p

AttributeError: module 'scipy.stats' has no attribute 'ttest_lsamp'

## 보고서 작성
 - t-통계량 / p-value 값

전 / 후 시각화 코드 / 전 평균값, 후 평균값

In [11]:
from scipy import stats

t_statistic, p_value = stats.ttest_rel(training_rel['후'], training_rel['전'])

print("t-통계량:", t_statistic)
print("p-값:", p_value)


t-통계량: 2.204154108716127
p-값: 0.04004419061842953


## 독립비교 t-검정

- 학생
 + A 는 인문
 + B 는 체육
- A 그룹, B 그룹 사이에 집중력 테스트의 평균에서 차이가 나지 않을까 생각하여 테스트를 함
- 이 데이터로부터 어떤검정을 수행해야 A 랑 B 의 집중력에 유의한 차이가 있는지 확인


In [13]:
#귀무가설: A 그룹과 B 그룹의 집중력 테스트의 평균은 같다.
#대립가설: A 그룹과 B 그룹의 집중력 테스트의 평균은 다르다.

# 독립비교 t-검정 두가지 가정
 #+1) 각 그룹 데이터의 정규성을 검정
 #+2) 두 그룹의 분산이 같다 (=등분산성 검정), levene 검정 시행


training_ind = pd.read_csv('/content/drive/MyDrive/data/ch11_training_ind.csv')

training_ind.head()

t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var = True) # 등분산성이 같다

t, p

(-1.7608157246524712, 0.08631600572195147)

- 결론은 두 그룹의 평균 차이는 없더라~ 귀무가설 채택

In [14]:
# Levene 검정 이후 보니 등분산성이 다르더라
t, p = stats.ttest_ind(training_ind['A'], training_ind['B'], equal_var = False)
t, p

(-1.760815724652471, 0.08695731107259361)

## 비모수 검정
- 모수 검정에서 각 가정을 만족시키지 못할 때, 비모수 검정을 씀

### 윌콕슨의 부호순위검정
- 대응표본에서 차이에 정규분포를 만족시키지 못할 때,
  + 평균차이 아님(x)
  + 중앙값 차이에 대한 검정 (ㅇ)
  + 중앙값 = 50%에 해당하는 값

In [16]:
training_rel = pd.read_csv('/content/drive/MyDrive/data/ch11_training_rel.csv')
toy_df = training_rel[:6].copy()
toy_df['차'] = toy_df['후'] - toy_df['전']
toy_df

,전,후,차
0,59,41,-18
1,52,63,11
2,55,68,13
3,61,59,-2
4,59,84,25
5,45,37,-8


In [18]:
# 순위 적용

diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,41,-18,5
1,52,63,11,3
2,55,68,13,4
3,61,59,-2,1
4,59,84,25,6
5,45,37,-8,2


In [19]:
import numpy as np

r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(8, 13)

In [21]:
toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
#toy_df

r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(0, 21)

In [22]:
toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,차,순위
0,59,60,1,1
1,52,50,-2,2
2,55,52,-3,3
3,61,65,4,4
4,59,64,5,5
5,45,39,-6,6
